# Data Cleaning

Often, we will have a DataFrame with the information that we need, but the data are not organized in the right way. We may need to substitute variables, handle missing data, calculate summary statistics, and rearrange data to get it in the right shape. Polars has many tools for doing these things.

## Reading data from the datascipsych package

The `datascipsych` package includes a `datasets` module that can be used to access datasets that are included with the package.

To use the `datascipsych` package, it must be first be installed. 

To install the package: (1) make sure you have a virtual environment and that it is activated; (2) open a terminal (with the virtual environment activated) and run `pip install -e .`.

This will install functions and datasets available in the `datascipsych` package into your environment.

First, we will use the `datascipsych.datasets` module to access the data for the Osth & Fox (2019) study. Note that the dataset is not stored in the current directory. Instead, we access the dataset from the installed `datascipsych` package. This allows us to access the same dataset, no matter what our current directory is.

In [1]:
from datascipsych import datasets
dataset_file = datasets.get_dataset_file("Osth2019")
print(dataset_file)

/Users/morton/VS Code/datascipsych/src/datascipsych/data/Osth2019.csv


The `dataset_file` should give the path to the dataset on your system.

If you get a `ModuleNotFoundError`, you do not have the `datascipsych` package installed. Follow the instructions given above to install the package.

Now, we can read the dataset file from the installed package. We'll make things simpler by selecting a subset of columns.

In [2]:
import polars as pl
raw = (
    pl.read_csv(dataset_file)
    .select("phase", "type", "word1", "word2", "response", "RT", "correct", "lag")
)

## Recoding variables

One relatively simple form of data cleaning is to take the data in a column and *recode* it. For example, a spreadsheet may label conditions with numeric codes that are hard to read and remember. Replacing these codes with strings makes the dataset easier to understand.

The Osth & Fox (2019) dataset has a number of -1 values in it. These -1 values indicate missing or undefined data. For example, during study trials, participants do not make any response. On these trials, the `response` column is set to -1 to indicate that no response was recorded.

In [3]:
raw.head()

phase,type,word1,word2,response,RT,correct,lag
str,str,str,str,i64,f64,i64,i64
"""study""","""intact""","""formal""","""positive""",-1,-1.0,-1,-1
"""study""","""intact""","""skin""","""careful""",-1,-1.0,-1,-1
"""study""","""intact""","""upon""","""miss""",-1,-1.0,-1,-1
"""study""","""intact""","""single""","""tradition""",-1,-1.0,-1,-1
"""study""","""intact""","""prove""","""airport""",-1,-1.0,-1,-1


We can replace the -1 values using the `replace` method. Instead of -1, we want missing values to be set to the special value `null`. Polars will automatically exclude those values from calculations. To set a value to `null` in the DataFrame, we can set it to `None` when calling `replace`.

In [4]:
raw.with_columns(
    pl.col("response").replace(-1, None)
).head()

phase,type,word1,word2,response,RT,correct,lag
str,str,str,str,i64,f64,i64,i64
"""study""","""intact""","""formal""","""positive""",null,-1.0,-1,-1
"""study""","""intact""","""skin""","""careful""",null,-1.0,-1,-1
"""study""","""intact""","""upon""","""miss""",null,-1.0,-1,-1
"""study""","""intact""","""single""","""tradition""",null,-1.0,-1,-1
"""study""","""intact""","""prove""","""airport""",null,-1.0,-1,-1


We used `with_columns` to keep all the existing columns. We used `pl.col("response")` to represent the `response` column, and then called the `replace` method. The first input indicates the value to replace, and the second input indicates the value to replace it with.

Note that there are multiple columns that follow the same convention, with -1 indicating missing values. We'll want to replace all of them.

In Polars, we can carry out the same expression on multiple columns using *expression expansion*. Instead of writing multiple expressions with `replace` (one for each column), we can write just one expression that should be applied to multiple columns.

In [5]:
raw.with_columns(
    pl.col("response", "RT", "correct", "lag").replace(-1, None)
).head()

phase,type,word1,word2,response,RT,correct,lag
str,str,str,str,i64,f64,i64,i64
"""study""","""intact""","""formal""","""positive""",null,null,null,null
"""study""","""intact""","""skin""","""careful""",null,null,null,null
"""study""","""intact""","""upon""","""miss""",null,null,null,null
"""study""","""intact""","""single""","""tradition""",null,null,null,null
"""study""","""intact""","""prove""","""airport""",null,null,null,null


Sometimes, it's helpful to recode a column to get different labels. The `type` column has two values: `"intact"` and `"rearranged"`. Say we want to translate this into more standard labels typically used in recognition memory experiments such as this, namely `"target"` and `"lure"`. We can do this by passing a dictionary to the `replace` method. This time, we'll create a new column called `probe_type` that has this alternate coding.

In [6]:
raw.with_columns(
    probe_type=pl.col("type").replace({"intact": "target", "rearranged": "lure"})
).head()

phase,type,word1,word2,response,RT,correct,lag,probe_type
str,str,str,str,i64,f64,i64,i64,str
"""study""","""intact""","""formal""","""positive""",-1,-1.0,-1,-1,"""target"""
"""study""","""intact""","""skin""","""careful""",-1,-1.0,-1,-1,"""target"""
"""study""","""intact""","""upon""","""miss""",-1,-1.0,-1,-1,"""target"""
"""study""","""intact""","""single""","""tradition""",-1,-1.0,-1,-1,"""target"""
"""study""","""intact""","""prove""","""airport""",-1,-1.0,-1,-1,"""target"""


Now that we have tested out the commands we want to run, let's put them all together to get a set of commands that will run all our recoding and assign the output to a new variable called `data`.

In [7]:
data = raw.with_columns(
    pl.col("response", "RT", "correct", "lag").replace(-1, None),
    probe_type=pl.col("type").replace({"intact": "target", "rearranged": "lure"})
)
data.head()

phase,type,word1,word2,response,RT,correct,lag,probe_type
str,str,str,str,i64,f64,i64,i64,str
"""study""","""intact""","""formal""","""positive""",null,null,null,null,"""target"""
"""study""","""intact""","""skin""","""careful""",null,null,null,null,"""target"""
"""study""","""intact""","""upon""","""miss""",null,null,null,null,"""target"""
"""study""","""intact""","""single""","""tradition""",null,null,null,null,"""target"""
"""study""","""intact""","""prove""","""airport""",null,null,null,null,"""target"""


Sometimes, a raw data file may code conditions as integers, but we want to convert the numeric condition codes into string labels.

In [8]:
simple = pl.DataFrame(
    {
        "trial": [1, 2, 3, 4],
        "condition": [1, 1, 2, 2],
    }
)
simple

trial,condition
i64,i64
1,1
2,1
3,2
4,2


We cannot use `replace` to change the type of a column. To recode integers as strings, we can first `cast` the integers as strings, using `.cast(pl.String)`. The `pl.String` attribute represents the string format that Polars uses.

In [9]:
simple.with_columns(
    condition=pl.col("condition").cast(pl.String).replace({"1": "target", "2": "lure"})
)

trial,condition
i64,str
1,"""target"""
2,"""target"""
3,"""lure"""
4,"""lure"""


## Handling missing data

Sometimes, observations will be missing, for example the response on a trial may be undefined if the participant did not make a response.

Let's set up an example DataFrame with some missing responses.

In [10]:
responses = pl.DataFrame(
    {
        "correct": [1, 0, 1, None, 0, 0, 1, None, 1],
        "response_time": [1.2, 2.1, 1.4, None, 1.8, 1.6, 1.4, None, 1.6],
    }
)
responses

correct,response_time
i64,f64
1,1.2
0,2.1
1,1.4
null,null
0,1.8
0,1.6
1,1.4
null,null
1,1.6


We can calculate the mean accuracy and response time using the `mean` function.

In [11]:
responses.mean()

correct,response_time
f64,f64
0.571429,1.585714


However, we might want to count missing responses as incorrect by coding them as 0. We can recode `null` values as something else using `fill_null`.

In [12]:
missing = responses.with_columns(
    correct_missing=pl.col("correct").fill_null(0)
)
missing.mean()

correct,response_time,correct_missing
f64,f64,f64
0.571429,1.585714,0.444444


Note that the accuracy is lower when we count missing responses as incorrect.

We can also use `filter` together with `is_not_null` to remove rows where a column is `null`.

In [13]:
included = responses.filter(pl.col("correct").is_not_null())
included

correct,response_time
i64,f64
1,1.2
0,2.1
1,1.4
0,1.8
0,1.6
1,1.4
1,1.6


## Grouping and aggregation

When we are analyzing a dataset, we usually don't just want to calculate summary statistics for a whole column of data. Often we will instead want to break the dataset into groups, and calculate the summary statistics for each group.

Grouping and aggregation allow us to calculate summary statistics for groups within our data using very little code. Here, we'll calculate the mean response time for correct trials (where `correct == 1`) and incorrect trials (where `correct == 0`).

In [14]:
included.group_by("correct").agg(pl.col("response_time").mean())

correct,response_time
i64,f64
0,1.833333
1,1.4


We first grouped the data to make a group for each value of the `correct` column. We then aggregated the data in the `response_time` column by calculating the mean within each group.

Let's break down this code to see how it works.

To perform aggregation, we start by calling `group_by`. Here, we give the name of the `correct` column to indicate that we want to make one group for each value of that column. There will be one group for `correct == 0` and one group for `correct == 1`.

In [15]:
g = included.group_by("correct")

Normally, we would next call the `agg` method to calculate some aggregation wtihin each group. Instead, we'll convert the `GroupBy` object into a list so we can look at it.

In [16]:
g = list(g)

Each group includes a `name` (the value for each group, which here is 0 or 1) and a DataFrame that just has the rows that match that value.

First, we have all the incorrect trials where `correct == 0`.

In [17]:
name, data = g[0]
print(name)
data

(0,)


correct,response_time
i64,f64
0,2.1
0,1.8
0,1.6


Next, we have all the correct trials where `correct == 1`.

In [18]:
name, data = g[1]
print(name)
data

(1,)


correct,response_time
i64,f64
1,1.2
1,1.4
1,1.4
1,1.6


We can aggregate the data within each group through multiple different methods. One simple method is to just pass the name of a column to the `agg` function. This will give us a column that has a list of all the observations that were in each group.

In [19]:
included.group_by("correct").agg(pl.col("response_time"))

correct,response_time
i64,list[f64]
1,"[1.2, 1.4, … 1.6]"
0,"[2.1, 1.8, 1.6]"


Often, we'll want to calculate a summary statistic, like the mean, for each group. We can do that by specifying the column to aggregate and calling the `mean` function.

In [20]:
included.group_by("correct").agg(pl.col("response_time").mean())

correct,response_time
i64,f64
0,1.833333
1,1.4


We can also use calculate other summary statistics, like the standard deviation.

In [21]:
included.group_by("correct").agg(pl.col("response_time").std())

correct,response_time
i64,f64
1,0.163299
0,0.251661


We can even calculate multiple summary statistics at the same time. In this case, we must give the columns different names to avoid getting an error. Like in `select` and `with_columns`, we can assign names using keyword arguments to `agg`.

In [22]:
(
    included.group_by("correct")
    .agg(
        mean=pl.col("response_time").mean(),
        std=pl.col("response_time").std(),
    )
)

correct,mean,std
i64,f64,f64
1,1.4,0.163299
0,1.833333,0.251661


## Reshaping data

Sometimes, we need to transform our dataset to a different shape to make it easier to work with.

Say that we have data in a study where trials are split into blocks. Each block is represented by one row in the DataFrame. The `1` column indicates if trial 1 was correct, the `2` column indicates if trial 2 was correct, and the `3` column indicates if trial 3 was correct.

| block | 1 | 2 | 3 |
| ----- | - | - | - |
| 1     | 1 | 0 | 0 |
| 2     | 0 | 1 | 0 |

This is called a *wide* format table. If we want to analyze the accuracy of trials within each block, we will have to access the correct column.

Often it is easier to work with *long* format instead. In a long-format DataFRame, there is one row for each set of observations. Here, that means there is one row for each trial.

| block | trial | correct |
| ----- | ----- | ------- |
| 1     | 1     | 1       |
| 1     | 2     | 0       |
| 1     | 3     | 0       |
| 2     | 1     | 0       |
| 2     | 2     | 1       |
| 2     | 3     | 0       |

In [23]:
df = pl.DataFrame(
    {
        "block": [1, 2],
        "1": [1, 0],
        "2": [0, 1],
        "3": [0, 0],
    }
)
df

block,1,2,3
i64,i64,i64,i64
1,1,0,0
2,0,1,0


In [24]:
df.unpivot(
    ["1", "2", "3"], 
    index="block",
    variable_name="trial",
    value_name="correct",
).with_columns(pl.col("trial").cast(pl.Int64)).sort("block")

block,trial,correct
i64,i64,i64
1,1,1
1,2,0
1,3,0
2,1,0
2,2,1
2,3,0
